## 上证综指

In [1]:
import sys
import talib as talib
import pandas as pd
import numpy as np
from pyecharts import options as opts
sys.path.append("..") 
from imp import reload
from src.ts_stock import TsStock
from src.chart import Chart

tsStock = TsStock();
chart = Chart();

/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [2]:
keyword = '上证综指'
sma_periods = [8,17,25,99,145]
hs_index = tsStock.get_daily_data(keyword,'20120101','')
hs_index = hs_index[['trade_date','close','open','high','low','vol']]
kdata_datas=np.array(hs_index.loc[:,['open','close','low','high']])
stock_info = tsStock.get_stock_info(keyword);
trade_dates = np.array(hs_index.trade_date)
vols = np.array(hs_index['vol'])
hs_index['sar'] = talib.SAR(hs_index.high, hs_index.low, acceleration=0.02, maximum=0.2)
hs_index['ATR']=talib.ATR(hs_index.high,hs_index.low,hs_index.close,timeperiod=25)
sma_dic= {}
for period in sma_periods:
    name = "SMA" + str(period)
    sma = talib.SMA(np.array(hs_index.close),period)
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] = sma.round(2).tolist();
    
sar = {}
sar['down'] = hs_index.loc[hs_index['close'] > hs_index['sar'],['trade_date','sar']]
sar['up'] = (hs_index[hs_index['close'] < hs_index['sar']])[['trade_date','sar']]

hs_index['macd_dif'],hs_index['macd_dem'],hs_index['macd_ocr'] = talib.MACD(hs_index.close,12,26,9)



grid_chart = chart.get_grid(stock_info,trade_dates.tolist(),kdata_datas.tolist(),sma_dic,vols.tolist())

macd = chart.get_macd_chart(
    trade_dates.tolist(),
    hs_index.macd_dif.to_numpy().round(2).tolist(),
    hs_index.macd_dem.to_numpy().round(2).tolist(),
    (hs_index["macd_ocr"]*2).to_numpy().round(2).tolist()
)

grid_chart.add(
    macd,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="75%", height="10%"
    ),
)


## 上证综指K线

In [3]:
# grid_chart.render_notebook()

## 个股

In [4]:
sma_periods = [8,17,25,99,145,418,453]
keyword = '601700'
start_day='20120101'
stock_info=tsStock.get_stock_info(keyword)
hs= tsStock.get_daily_data(keyword,start_day,'')
hs=hs[['trade_date','close','open','high','low','vol']]
kdatas = np.array(hs.loc[:,['open','close','low','high']])
trade_dates = np.array(hs.trade_date)
vols = np.array(hs['vol'])
hs['sar'] = talib.SAR(hs.high, hs.low, acceleration=0.1, maximum=2)

sma_dic= {}
for period in sma_periods:
    name = "SMA" + str(period)
    sma = talib.SMA(np.array(hs.close),period)
    hs[name] =sma;
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] = sma.round(2).tolist();
sar = {}
sar['down'] = hs.loc[hs['close'] > hs['sar'],['trade_date','sar']]
sar['up'] = (hs[hs['close'] < hs['sar']])[['trade_date','sar']]

grid_chart= chart.get_grid(stock_info,trade_dates.tolist(), kdatas.tolist(),sma_dic,vols.tolist(), sar=sar)
    
hs['macd_dif'],hs['macd_dem'],hs['macd_ocr'] = talib.MACD(hs.close,12,26,9)

macd = chart.get_macd_chart(
    trade_dates.tolist(),
    hs.macd_dif.to_numpy().round(2).tolist(),
    hs.macd_dem.to_numpy().round(2).tolist(),
    (hs["macd_ocr"]*2).to_numpy().round(2).tolist()
)

grid_chart.add(
    macd,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="75%", height="10%"
    ),
)


## k

In [5]:
grid_chart.render_notebook()